In [1]:
'''Import packages and set options'''
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import pickle

pd.set_option('display.max_columns', None)

In [ ]:
'''Declare city and state variables'''
city = 'Wilmington'
state = 'NC'
search_string = city + ', ' + state

In [17]:
'''Get API Keys'''
api_keys = pd.read_csv('api_key.csv')
rapid_api_key = api_keys.loc[api_keys['API'] == 'rapid']['KEY'][0]

In [ ]:
'''Set search parameters'''
home_type = "Houses"
maxPrice = "400000"
sqftMin = "1000"
status_type = "RecentlySold"
page = "1"

In [ ]:
'''Get data from Zillow'''
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

for_sale_querystring = {"location":search_string,
                        "page":page,
                        "home_type":home_type,
                        "maxPrice":maxPrice,
                        "sqftMin":sqftMin}

sold_querystring = {"location":search_string,
                    "page":page,
                    "status_type":status_type,
                    "home_type":home_type,
                    "maxPrice":maxPrice,
                    "sqftMin":sqftMin}


headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

for_sale_response = requests.request("GET", url, headers=headers, params=for_sale_querystring)
for_sale_json = for_sale_response.json()
num_page = for_sale_json['totalPages']
if num_page > 1:
    for i in range(1, num_page):
        page = int(page)
        page += 1
        for_sale_querystring.update({"page":str(page)})
        temp_response = requests.request("GET", url, headers=headers, params=for_sale_querystring)
        temp_json = temp_response.json()
        for_sale_json['props'].extend(temp_json['props'])
        
sold_response = requests.request("GET", url, headers=headers, params=sold_querystring)
sold_json = sold_response.json()
num_page = sold_json['totalPages']
page="1"
if num_page > 1:
    for i in range(1, num_page):
        page = int(page)
        page += 1
        sold_querystring.update({"page":str(page)})
        temp_response = requests.request("GET", url, headers=headers, params=sold_querystring)
        temp_json = temp_response.json()
        time.sleep(20)
        #print(temp_json)
        sold_json['props'].extend(temp_json['props'])

In [ ]:
len(sold_json['props'])

In [ ]:
'''Pickle datafile'''
with open('for_sale.pkl', 'wb') as f:
    pickle.dump(for_sale_json, f)
    
with open('sold.pkl', 'wb') as f:
    pickle.dump(sold_json, f)

In [2]:
'''Open pickle file'''
with open('for_sale.pkl', 'rb') as f:
    for_sale_json = pickle.load(f)

with open('sold.pkl', 'rb') as f:
    sold_json = pickle.load(f)

In [3]:
'''Convert output to dataframe'''
for_sale_df = pd.json_normalize(data = for_sale_json['props'])
sold_df = pd.json_normalize(data = sold_json['props'])

In [4]:
sold_df

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,unit,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,newConstructionType,listingSubType.is_newHome,listingSubType.is_openHouse
0,1661410800000,SINGLE_FAMILY,9147.60,"321 Tributary Circle Lot 37, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/5c8ee4633da...,349990,3.0,Lot 37,NaN,NaN,RECENTLY_SOLD,2065410697,None,-1,2.0,1475,USA,USD,sqft,True,NaN,NaN,NaN,NaN
1,1661410800000,SINGLE_FAMILY,7405.20,"2900 Miranda Court, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/42d46ad50d6...,280000,3.0,NaN,-77.881836,34.302300,RECENTLY_SOLD,54283198,None,-1,2.0,1325,USA,USD,sqft,True,NaN,NaN,NaN,NaN
2,1661410800000,SINGLE_FAMILY,10454.40,"3005 Weatherby Court, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/d9e705bb482...,330000,2.0,NaN,-77.878170,34.303795,RECENTLY_SOLD,54283256,None,-1,2.0,1378,USA,USD,sqft,True,NaN,NaN,NaN,NaN
3,1661410800000,SINGLE_FAMILY,9147.60,"6701 Batsonwood Court, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/2d4fe99952c...,205000,2.0,NaN,-77.840220,34.266884,RECENTLY_SOLD,54294303,None,-1,2.0,1110,USA,USD,sqft,True,NaN,NaN,NaN,NaN
4,1661410800000,SINGLE_FAMILY,0.57,"3437 Sea Eagle Court, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/8d68f5ac7b5...,369000,3.0,NaN,-77.877440,34.181217,RECENTLY_SOLD,54332953,None,-1,2.0,1305,USA,USD,acres,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1650438000000,SINGLE_FAMILY,3049.20,"712 Varsity Drive, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/38b58a748cf...,232000,3.0,NaN,-77.887560,34.222576,RECENTLY_SOLD,54314963,None,-1,2.0,1070,USA,USD,sqft,True,NaN,NaN,NaN,NaN
796,1650438000000,SINGLE_FAMILY,10019.00,"302 Lancaster Rd, Wilmington, NC 28409",https://maps.googleapis.com/maps/api/staticmap...,122500,3.0,NaN,-77.876710,34.191635,RECENTLY_SOLD,54325704,None,-1,1.5,1056,USA,USD,sqft,None,NaN,NaN,NaN,NaN
797,1650438000000,SINGLE_FAMILY,9583.00,"310 Westchester Rd, Wilmington, NC 28409",https://maps.googleapis.com/maps/api/staticmap...,145000,3.0,NaN,-77.878235,34.190790,RECENTLY_SOLD,54325758,None,-1,1.5,1426,USA,USD,sqft,None,NaN,NaN,NaN,NaN
798,1650438000000,SINGLE_FAMILY,0.42,"1009 Francis Marion Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/73168e11175...,277800,3.0,NaN,-77.925230,34.190376,RECENTLY_SOLD,54330160,None,-1,2.0,1385,USA,USD,acres,True,NaN,NaN,NaN,NaN


In [5]:
'''Check difference between two tables'''
difference = list(set(for_sale_df.columns) - set(sold_df.columns))
difference

[]

In [6]:
for_sale_df

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome,unit
0,None,SINGLE_FAMILY,0.46,"134 Whipporwill Lane, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/aaea5a8fd22...,375000,3,-77.865320,34.153725,FOR_SALE,54340660,None,-1,2,1644.0,USA,USD,acres,True,True,True,NaN,NaN,NaN
1,None,SINGLE_FAMILY,8276.40,"1301 Kornegay Avenue, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/a83a8968903...,270000,4,-77.910420,34.253407,FOR_SALE,54299281,None,-1,2,1886.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN
2,None,SINGLE_FAMILY,0.57,"669 Hidden Valley Road, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/587c523a4e5...,359900,3,-77.891650,34.151524,FOR_SALE,54339572,None,-1,3,1788.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
3,None,SINGLE_FAMILY,0.32,"4810 W Rye Lane, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/e8768e200a7...,300000,3,-77.867990,34.266224,FOR_SALE,54293012,None,-1,2,1096.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
4,None,SINGLE_FAMILY,6969.60,"4660 Runaway Bay Lane, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/fc68661be25...,369900,3,-77.864494,34.271942,FOR_SALE,331476698,None,-1,2,1531.0,USA,USD,sqft,True,True,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,None,SINGLE_FAMILY,NaN,"KERRY Plan, Riverside",https://photos.zillowstatic.com/fp/851831bbcae...,359990,3,-77.927040,34.288140,FOR_SALE,2079470488,None,-1,2,1475.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,True,# HEDH4V
79,None,SINGLE_FAMILY,NaN,"ARIA Plan, Riverside",https://photos.zillowstatic.com/fp/52f2d1a574e...,372990,3,-77.927040,34.288140,FOR_SALE,2079470486,None,-1,2,1618.0,USA,USD,None,True,NaN,True,BUILDER_PLAN,True,# UD84Q1
80,None,SINGLE_FAMILY,NaN,"Steel Creek Plan, Del Webb Wilmington",https://photos.zillowstatic.com/fp/d1ec64fffd5...,372990,2,-77.935920,34.151676,FOR_SALE,2077429102,None,-1,2,1355.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,True,# GQ8OL
81,None,SINGLE_FAMILY,NaN,"Harbor Plan, Scotts Hill Village",https://photos.zillowstatic.com/fp/c463cd4feee...,399990,3,-77.763985,34.316654,FOR_SALE,2061894741,None,-1,3,2025.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,True,# LBLMB4


In [7]:
'''Drop columns that are not in both frames'''
for_sale_df.drop(difference, axis =1, inplace=True)

In [8]:
'''Concatenate dataframes into frame'''
frames = [for_sale_df, sold_df]
all_sales = pd.concat(frames)
all_sales

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome,unit
0,None,SINGLE_FAMILY,0.46,"134 Whipporwill Lane, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/aaea5a8fd22...,375000,3.0,-77.865320,34.153725,FOR_SALE,54340660,None,-1,2.0,1644.0,USA,USD,acres,True,True,True,NaN,NaN,NaN
1,None,SINGLE_FAMILY,8276.40,"1301 Kornegay Avenue, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/a83a8968903...,270000,4.0,-77.910420,34.253407,FOR_SALE,54299281,None,-1,2.0,1886.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN
2,None,SINGLE_FAMILY,0.57,"669 Hidden Valley Road, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/587c523a4e5...,359900,3.0,-77.891650,34.151524,FOR_SALE,54339572,None,-1,3.0,1788.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
3,None,SINGLE_FAMILY,0.32,"4810 W Rye Lane, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/e8768e200a7...,300000,3.0,-77.867990,34.266224,FOR_SALE,54293012,None,-1,2.0,1096.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
4,None,SINGLE_FAMILY,6969.60,"4660 Runaway Bay Lane, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/fc68661be25...,369900,3.0,-77.864494,34.271942,FOR_SALE,331476698,None,-1,2.0,1531.0,USA,USD,sqft,True,True,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1650438000000,SINGLE_FAMILY,3049.20,"712 Varsity Drive, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/38b58a748cf...,232000,3.0,-77.887560,34.222576,RECENTLY_SOLD,54314963,None,-1,2.0,1070.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN
796,1650438000000,SINGLE_FAMILY,10019.00,"302 Lancaster Rd, Wilmington, NC 28409",https://maps.googleapis.com/maps/api/staticmap...,122500,3.0,-77.876710,34.191635,RECENTLY_SOLD,54325704,None,-1,1.5,1056.0,USA,USD,sqft,None,NaN,NaN,NaN,NaN,NaN
797,1650438000000,SINGLE_FAMILY,9583.00,"310 Westchester Rd, Wilmington, NC 28409",https://maps.googleapis.com/maps/api/staticmap...,145000,3.0,-77.878235,34.190790,RECENTLY_SOLD,54325758,None,-1,1.5,1426.0,USA,USD,sqft,None,NaN,NaN,NaN,NaN,NaN
798,1650438000000,SINGLE_FAMILY,0.42,"1009 Francis Marion Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/73168e11175...,277800,3.0,-77.925230,34.190376,RECENTLY_SOLD,54330160,None,-1,2.0,1385.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN


In [9]:
'''Check datatypes'''
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883 entries, 0 to 799
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   dateSold                     800 non-null    object 
 1   propertyType                 883 non-null    object 
 2   lotAreaValue                 870 non-null    float64
 3   address                      883 non-null    object 
 4   imgSrc                       883 non-null    object 
 5   price                        883 non-null    int64  
 6   bedrooms                     876 non-null    float64
 7   longitude                    825 non-null    float64
 8   latitude                     825 non-null    float64
 9   listingStatus                883 non-null    object 
 10  zpid                         883 non-null    object 
 11  contingentListingType        0 non-null      object 
 12  daysOnZillow                 883 non-null    int64  
 13  bathrooms           

In [10]:
'''Convert dateSold to datetime'''
all_sales['dateSold']= pd.to_datetime(all_sales['dateSold'])
all_sales['dateSold'] = pd.to_datetime(all_sales['dateSold'].dt.strftime('%Y-%m'))

In [11]:
'''Coordinate box of desired living area'''
UL = 34.250170, -77.946410
UR = 34.243651, -77.915227
LL = 34.225592, -77.944480
LR = 34.226329, -77.911553

print(UL[1])

-77.94641


In [12]:
'''Define desired geographic area'''
relevant_homes = all_sales.query("longitude >= @UL[1] and longitude <= @LR[1] and latitude <= @UL[0] and latitude >= @LR[0]")
relevant_homes

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome,unit
12,NaT,SINGLE_FAMILY,4791.600000,"105 N 13th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0b515c83515...,239900,3.0,-77.933334,34.238210,FOR_SALE,54301987,None,-1,2.0,1463.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN
18,NaT,SINGLE_FAMILY,3484.800000,"111 N 8th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0a956c1f318...,299900,3.0,-77.939790,34.237823,FOR_SALE,54301687,None,-1,1.0,1292.0,USA,USD,sqft,True,True,True,NaN,NaN,NaN
19,NaT,SINGLE_FAMILY,0.340000,"826 Colonial Drive, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/46819c4e07d...,395000,2.0,-77.912620,34.226948,FOR_SALE,54311633,None,-1,2.0,1541.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
24,NaT,SINGLE_FAMILY,3920.400000,"914 Queen Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/98a9184bfeb...,324900,3.0,-77.936200,34.227203,FOR_SALE,54311353,None,-1,2.0,1139.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN
30,NaT,SINGLE_FAMILY,0.250000,"449 N 21st Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/608ccf33669...,315000,3.0,-77.924840,34.244804,FOR_SALE,54300922,None,-1,2.0,1366.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,1970-01-01,SINGLE_FAMILY,9583.200000,"2211 Brandon Road, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/3bbbad14f28...,355000,3.0,-77.922240,34.245636,PENDING,54299931,None,-1,1.0,1217.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN
761,1970-01-01,SINGLE_FAMILY,5998.000000,"1019 N 11th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,198000,2.0,-77.935640,34.248203,RECENTLY_SOLD,69142067,None,-1,1.5,1151.0,USA,USD,sqft,None,NaN,NaN,NaN,NaN,NaN
783,1970-01-01,SINGLE_FAMILY,5227.200000,"405 N 10th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/00d5b11f672...,330000,3.0,-77.937690,34.240840,RECENTLY_SOLD,54300801,None,-1,2.0,1609.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN
792,1970-01-01,SINGLE_FAMILY,0.303788,"521 Kenwood Ave, Wilmington, NC 28405",https://maps.googleapis.com/maps/api/staticmap...,180000,2.0,-77.917700,34.246040,RECENTLY_SOLD,54300018,None,-1,1.5,1745.0,USA,USD,acres,None,NaN,NaN,NaN,NaN,NaN


In [13]:
'''Save file as csv'''
relevant_homes.to_csv('relevant_homes.csv')
relevant_homes = pd.read_csv('relevant_homes.csv')

In [14]:
'''Calculate Price per sqft'''
relevant_homes.loc[relevant_homes['lotAreaUnit'] == 'sqft', 'Price/sqft'] = relevant_homes['price']/relevant_homes['livingArea']
relevant_homes.sort_values('Price/sqft')

,Unnamed: 0,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome,unit,Price/sqft
65,450,1970-01-01,SINGLE_FAMILY,4356.000000,"1705 Queen St, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/af7495216e7...,19000,3.0,-77.926060,34.228916,RECENTLY_SOLD,54311559,NaN,-1,1.00,1075.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,17.674419
80,583,1970-01-01,SINGLE_FAMILY,3301.000000,"1604 Castle St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,40000,3.0,-77.927520,34.229218,RECENTLY_SOLD,69139418,NaN,-1,1.00,1387.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,28.839221
31,150,1970-01-01,SINGLE_FAMILY,4791.600000,"18 N 5th Ave, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/4917c8cd7fc...,150500,0.0,-77.943110,34.236670,RECENTLY_SOLD,54301716,NaN,-1,2.00,2302.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,65.377932
38,213,1970-01-01,SINGLE_FAMILY,1981.000000,"516 Brunswick St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,85000,2.0,-77.943344,34.244570,RECENTLY_SOLD,54300209,NaN,-1,1.00,1012.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,83.992095
59,430,1970-01-01,SINGLE_FAMILY,3410.000000,"708 S 10th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,115000,3.0,-77.935950,34.227108,RECENTLY_SOLD,54311351,NaN,-1,2.00,1136.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,101.232394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,435,1970-01-01,SINGLE_FAMILY,1.166690,"409 S 5th Ave, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,8500,NaN,-77.942210,34.230190,RECENTLY_SOLD,84713988,NaN,-1,2.25,26149.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,449,1970-01-01,SINGLE_FAMILY,0.300000,"2012 Klein Rd,",https://photos.zillowstatic.com/fp/e1e23b4b472...,3550,3.0,-77.928085,34.247543,RECENTLY_SOLD,54299788,NaN,-1,2.50,1218.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN
69,480,1970-01-01,SINGLE_FAMILY,0.300000,"2613 Willow Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/46700ca6b27...,291500,3.0,-77.913216,34.242874,RECENTLY_SOLD,54301235,NaN,-1,1.00,1232.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN
76,548,1970-01-01,SINGLE_FAMILY,0.270983,"2402 Shirley Rd, Wilmington, NC 28405",https://maps.googleapis.com/maps/api/staticmap...,250000,3.0,-77.917030,34.246147,RECENTLY_SOLD,54300012,NaN,-1,2.00,1688.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
'''Add value column'''
condition_list = [
    relevant_homes['Price/sqft'] >= 175,
    (relevant_homes['Price/sqft'] < 175) & (relevant_homes['Price/sqft'] >= 150),
    relevant_homes['Price/sqft'] < 150
]

choice_list = ['Expensive', 'Reasonable', 'Affordable']

relevant_homes['value'] = np.select(condition_list, choice_list)
relevant_homes

,Unnamed: 0,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome,unit,Price/sqft,value
0,12,NaN,SINGLE_FAMILY,4791.600000,"105 N 13th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0b515c83515...,239900,3.0,-77.933334,34.238210,FOR_SALE,54301987,NaN,-1,2.0,1463.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,163.978127,Reasonable
1,18,NaN,SINGLE_FAMILY,3484.800000,"111 N 8th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0a956c1f318...,299900,3.0,-77.939790,34.237823,FOR_SALE,54301687,NaN,-1,1.0,1292.0,USA,USD,sqft,True,True,True,NaN,NaN,NaN,232.120743,Expensive
2,19,NaN,SINGLE_FAMILY,0.340000,"826 Colonial Drive, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/46819c4e07d...,395000,2.0,-77.912620,34.226948,FOR_SALE,54311633,NaN,-1,2.0,1541.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,0
3,24,NaN,SINGLE_FAMILY,3920.400000,"914 Queen Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/98a9184bfeb...,324900,3.0,-77.936200,34.227203,FOR_SALE,54311353,NaN,-1,2.0,1139.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,285.250219,Expensive
4,30,NaN,SINGLE_FAMILY,0.250000,"449 N 21st Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/608ccf33669...,315000,3.0,-77.924840,34.244804,FOR_SALE,54300922,NaN,-1,2.0,1366.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,746,1970-01-01,SINGLE_FAMILY,9583.200000,"2211 Brandon Road, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/3bbbad14f28...,355000,3.0,-77.922240,34.245636,PENDING,54299931,NaN,-1,1.0,1217.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,291.700904,Expensive
92,761,1970-01-01,SINGLE_FAMILY,5998.000000,"1019 N 11th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,198000,2.0,-77.935640,34.248203,RECENTLY_SOLD,69142067,NaN,-1,1.5,1151.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,172.024327,Reasonable
93,783,1970-01-01,SINGLE_FAMILY,5227.200000,"405 N 10th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/00d5b11f672...,330000,3.0,-77.937690,34.240840,RECENTLY_SOLD,54300801,NaN,-1,2.0,1609.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,205.096333,Expensive
94,792,1970-01-01,SINGLE_FAMILY,0.303788,"521 Kenwood Ave, Wilmington, NC 28405",https://maps.googleapis.com/maps/api/staticmap...,180000,2.0,-77.917700,34.246040,RECENTLY_SOLD,54300018,NaN,-1,1.5,1745.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [18]:
'''Set up Google Maps API'''

google_api_key = api_keys.loc[api_keys['API'] == 'google']['KEY'][1]
google_api_key
gmaps.configure(api_key=google_api_key)

In [30]:
'''Create Google Map of Wilmington, NC'''
wilmington_coordinates = (34.236509, -77.933831)
fig = gmaps.figure(center=wilmington_coordinates, zoom_level=14, map_type = 'ROADMAP')

In [31]:
'''Get house locations'''
houses = relevant_homes[['latitude', 'longitude', 'value', 'address', 'imgSrc', 'price','Price/sqft', 'listingStatus']]

info_box_template = """
<dl>
<dt>Address</dt><dd>{address}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Price/sqft</dt><dd>{Price/sqft}</dd>
<dt>Status</dt><dd>{listingStatus}</dd>
<dt>Pictures</dt><dd><a href={imgSrc} target="_blank">Photos</a></dd>
</dl>
"""

affordable_houses = houses.loc[houses['value'] == 'Affordable']
affordable_location = affordable_houses[['latitude', 'longitude']]
affordable_house_info = [info_box_template.format(**house) for house in affordable_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

expensive_houses = houses.loc[houses['value'] == 'Expensive']
expensive = expensive[['latitude', 'longitude']]
expensive_house_info = [info_box_template.format(**house) for house in expensive_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

reasonable_houses = houses.loc[houses['value'] == 'Reasonable']
reasonable = reasonable[['latitude', 'longitude']]
reasonable_house_info = [info_box_template.format(**house) for house in reasonable_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

affordable_layer = gmaps.symbol_layer(affordable_location, fill_color="green", stroke_color="green", scale=6, info_box_content=affordable_house_info)
expensive_layer = gmaps.symbol_layer(expensive, fill_color='red', stroke_color='red', scale=6, info_box_content=expensive_house_info)
reasonable_layer = gmaps.symbol_layer(reasonable, fill_color='yellow', stroke_color='yellow', scale=6, info_box_content=reasonable_house_info)

fig.add_layer(affordable_layer)
fig.add_layer(expensive_layer)
fig.add_layer(reasonable_layer)
fig

Figure(layout=FigureLayout(height='420px'))